In [1]:
import numpy as np
import pandas as pd
import sklearn
from sklearn.model_selection import train_test_split, KFold, GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score

In [2]:
data = pd.read_csv('Housing.csv')
data.shape

(545, 13)

In [3]:
data.head()

,price,area,bedrooms,bathrooms,stories,mainroad,guestroom,basement,hotwaterheating,airconditioning,parking,prefarea,furnishingstatus
0,13300000,7420,4,2,3,yes,no,no,no,yes,2,yes,furnished
1,12250000,8960,4,4,4,yes,no,no,no,yes,3,no,furnished
2,12250000,9960,3,2,2,yes,no,yes,no,no,2,yes,semi-furnished
3,12215000,7500,4,2,2,yes,no,yes,no,yes,3,yes,furnished
4,11410000,7420,4,1,2,yes,yes,yes,no,yes,2,no,furnished


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 545 entries, 0 to 544
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   price             545 non-null    int64 
 1   area              545 non-null    int64 
 2   bedrooms          545 non-null    int64 
 3   bathrooms         545 non-null    int64 
 4   stories           545 non-null    int64 
 5   mainroad          545 non-null    object
 6   guestroom         545 non-null    object
 7   basement          545 non-null    object
 8   hotwaterheating   545 non-null    object
 9   airconditioning   545 non-null    object
 10  parking           545 non-null    int64 
 11  prefarea          545 non-null    object
 12  furnishingstatus  545 non-null    object
dtypes: int64(6), object(7)
memory usage: 55.5+ KB


In [5]:
# identify numerical and categorical variables
categorical_vars = data.select_dtypes(include='object').columns
numerical_vars = data.select_dtypes(include='int64').columns

In [6]:
# handle binary variables
for var in categorical_vars:
    if var != 'furnishingstatus':
        data[var] = data[var].map({'no': 0, 'yes': 1})

In [7]:
categorical_vars = data.select_dtypes(include='object').columns

In [8]:
# handle categorical variables
data = pd.get_dummies(data, columns=categorical_vars, drop_first=True, dtype='int64')

In [9]:
data.head()

,price,area,bedrooms,bathrooms,stories,mainroad,guestroom,basement,hotwaterheating,airconditioning,parking,prefarea,furnishingstatus_semi-furnished,furnishingstatus_unfurnished
0,13300000,7420,4,2,3,1,0,0,0,1,2,1,0,0
1,12250000,8960,4,4,4,1,0,0,0,1,3,0,0,0
2,12250000,9960,3,2,2,1,0,1,0,0,2,1,1,0
3,12215000,7500,4,2,2,1,0,1,0,1,3,1,0,0
4,11410000,7420,4,1,2,1,1,1,0,1,2,0,0,0


In [10]:
train_data, test_data = train_test_split(data, train_size=0.7, test_size=0.3, random_state=100)
train_data.shape, test_data.shape

((381, 14), (164, 14))

In [11]:
X_train = train_data.drop(columns='price', axis=1)
y_train = train_data['price']

X_test = test_data.drop(columns='price', axis=1)
y_test = test_data['price']

In [12]:
folds = KFold(n_splits=5, shuffle=True, random_state=100)
params = {
    'max_depth': [3,4,5,6,7],
    'n_estimators': [100, 120, 140, 170, 200, 225, 250],
    'min_samples_leaf': [3,5,7,10]
}

In [13]:
rfr = RandomForestRegressor(n_jobs=-1, random_state=100)
grid_search = GridSearchCV(rfr, params, cv=folds, n_jobs=-1, scoring='r2', verbose=1, return_train_score=True)

In [14]:
grid_search.fit(X_train, y_train)

Fitting 5 folds for each of 140 candidates, totalling 700 fits


GridSearchCV(cv=KFold(n_splits=5, random_state=100, shuffle=True),
             estimator=RandomForestRegressor(n_jobs=-1, random_state=100),
             n_jobs=-1,
             param_grid={'max_depth': [3, 4, 5, 6, 7],
                         'min_samples_leaf': [3, 5, 7, 10],
                         'n_estimators': [100, 120, 140, 170, 200, 225, 250]},
             return_train_score=True, scoring='r2', verbose=1)

In [15]:
rfr_best = grid_search.best_estimator_
rfr_best

RandomForestRegressor(max_depth=7, min_samples_leaf=7, n_jobs=-1,
                      random_state=100)

In [16]:
rfr_best.fit(X_train, y_train)

RandomForestRegressor(max_depth=7, min_samples_leaf=7, n_jobs=-1,
                      random_state=100)

In [17]:
y_train_pred = rfr_best.predict(X_train)

In [18]:
r2_score(y_train, y_train_pred)

0.7152114582638158

In [19]:
y_test_pred = rfr_best.predict(X_test)

In [20]:
r2_score(y_test, y_test_pred)

0.601666258321657